# Ch 02

토큰화 > 임베딩에 주 목적

## 1) 단어 추출

In [1]:
with open('The Verdict.txt', 'r') as f:
    raw_text = f.read()

print(len(raw_text))
print(raw_text[:400])

20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)

"The height of his glory"--that was what the women called it. I can hear


In [2]:
import re

# 공백 기준 분리 + 특수 문자를 별도 토큰으로 추출
# -- 는 하나의 토큰, 나머지 특수 문자는 개별 토큰
tokens = re.findall(r'--|[^\s,.\-:;?_!"()`\']+|[,.:;?_!"()`\']', raw_text)

print(f"토큰 수: {len(tokens)}")
print(tokens[:20])

토큰 수: 4728
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was']


## 2.3 토큰을 토큰 ID 로 변환

- 어휘 사전(vocabulary) 구축 필요

In [3]:
all_words = sorted(set(tokens))
vocab_size = len(all_words)
vocab = {token: i for i, token in enumerate(all_words)}

for i, item in enumerate(vocab.items()):
    print(item)
    if (i >= 50):
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [4]:
# 텍스트 토크나이저 구현

class SimpleTokenizerV1:
    # 특수 문자 목록
    special_tokens = {'--', ',', '.', ':', ';', '?', '_', '!', '"', '(', ')', '`', "'"}

    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.findall(r'--|[^\s,.\-:;?_!"()`\']+|[,.:;?_!"()`\']', text)
        ids = [self.str_to_int[token] for token in preprocessed if token in self.str_to_int]
        return ids

    def decode(self, ids):
        tokens = [self.int_to_str[id] for id in ids]
        # 첫 토큰은 그대로, 이후 단어 앞에만 공백 추가
        # 특수 문자 앞에는 공백 없음, -- 뒤에는 공백 없음
        result = []
        for i, token in enumerate(tokens):
            if i == 0 or token in self.special_tokens or tokens[i - 1] == '--':
                result.append(token)
            else:
                result.append(' ' + token)
        return ''.join(result)

In [5]:
# encode test
tokenizer = SimpleTokenizerV1(vocab)

ids = tokenizer.encode("Never think of it, my dear fellow--any more than if I'd never touched a brush.")

print(ids)
print(tokenizer.decode(ids))

[69, 1007, 726, 588, 5, 701, 325, 440, 6, 161, 689, 995, 569, 53, 2, 315, 711, 1032, 115, 236, 7]
Never think of it, my dear fellow--any more than if I' d never touched a brush.


## 2.4 특수 문맥 토큰 추가하기

비식별 문자에 대해 토큰 남기는 것
- `<|unk|>` : 사전에 없는 정보 토큰 대체
- `<|endoftext|>` : 관련이 없는 텍스트 사이 대체 토큰

In [6]:
all_tokens = sorted(set(tokens))
all_tokens.extend(['<|endoftext|>', '<|unk|>'])
vocab = {token: i for i, token in enumerate(all_tokens)}

print(f"토큰 수: {len(vocab)}")
print(list(vocab.items())[-5:])

토큰 수: 1141
[('younger', 1136), ('your', 1137), ('yourself', 1138), ('<|endoftext|>', 1139), ('<|unk|>', 1140)]


In [7]:
class SimpleTokenizerV2:
    # 특수 문자 목록
    special_tokens = {'--', ',', '.', ':', ';', '?', '_', '!', '"', '(', ')', '`', "'"}

    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.findall(r'--|[^\s,.\-:;?_!"()`\']+|[,.:;?_!"()`\']', text)
        preprocessed = [item if item in self.str_to_int else '<|unk|>' for item in preprocessed]
        ids = [self.str_to_int[token] for token in preprocessed if token in self.str_to_int]
        return ids

    def decode(self, ids):
        tokens = [self.int_to_str[id] for id in ids]
        # 첫 토큰은 그대로, 이후 단어 앞에만 공백 추가
        # 특수 문자 앞에는 공백 없음, -- 뒤에는 공백 없음
        result = []
        for i, token in enumerate(tokens):
            if i == 0 or token in self.special_tokens or tokens[i - 1] == '--':
                result.append(token)
            else:
                result.append(' ' + token)
        return ''.join(result)

In [8]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join([text1, text2])
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [9]:
tokenizer = SimpleTokenizerV2(vocab)

ids = tokenizer.encode(text)
print(ids)
print(tokenizer.decode(ids))

[1140, 5, 361, 1135, 630, 984, 10, 1139, 55, 997, 965, 993, 726, 997, 1140, 7]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


추가 특수 토큰
- `[BOS]` (beginning of sequence)
- `[EOS]` (end of sequence)
- `[PAD]` (padding)


## 2.5 바이트 페어 인코딩

- `<|unk|>` 토큰 사용 대신 단어를 부분 단어로 분할하는 byte pair encoding(BPE) 토크나이저를 사용
- BPE 는 구현이 복잡해 `tiktoken` 라이브러리 사용 예정
- 잘 알지 못하는 단어는 개별 문자 or 부분 문자로 토큰화 함

In [10]:
from importlib.metadata import version
import tiktoken

print(version('tiktoken'))

0.12.0


In [11]:
tokenizer = tiktoken.get_encoding("gpt2")

text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunkownPlace."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 2954, 593, 27271, 13]


In [12]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunkownPlace.


In [13]:
## 연습문제 2.1
sample_text = "Akwirw ier"
encoded = tokenizer.encode(sample_text)
decoded = tokenizer.decode(encoded)
print(decoded, encoded)

Akwirw ier [33901, 86, 343, 86, 220, 959]


## 2.6 슬라이딩 윈도로 데이터 샘플링하기

- 이때까지 토크나이징, 문자 임베딩을 살펴봄
- 이제 학습용으로 입력-타겟 쌍을 생성 해야함
- LLM 은 텍스트에 있는 다음 단어를 예측하는 식으로 사전 훈련 됨
- 슬라이딩 윈도우로 하나씩 데이터를 쪼개서 만들어 볼 예정
- torch Dataset, DataLoader 는 부록 A - 1.3 절 참고
- batch_size 는 LLM 학습에서 하이퍼 파라미터라고 불림
  - batch_size 가 커질수록 메모리는 많이 차지하지만, 데이터 잡음이 조금씩 줄을 수 있음
  - trade-off: 배치 크기에 대해서도 튜닝 가능한 옵션 > 하이퍼 파라미터라 부름
- 배치 사이에 중첩이 있으면 과대적합(overfitting) 이 증가할 수 있는데, 최대한 input 데이터 중첩을 피해서 과대적합을 방지할 수 있음

In [14]:
tokenizer = tiktoken.get_encoding("gpt2")

with open('The Verdict.txt', 'r') as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))
print(enc_text[:100])

5145
[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138, 257, 7026, 15632, 438, 2016, 257, 922, 5891, 1576, 438, 568, 340, 373, 645, 1049, 5975, 284, 502, 284, 3285, 326, 11, 287, 262, 6001, 286, 465, 13476, 11, 339, 550, 5710, 465, 12036, 11, 6405, 257, 5527, 27075, 11, 290, 4920, 2241, 287, 257, 4489, 64, 319, 262, 34686, 41976, 13, 357, 10915, 314, 2138, 1807, 340, 561, 423, 587, 10598, 393, 28537, 2014, 198, 198, 1, 464, 6001, 286, 465, 13476, 1, 438, 5562, 373, 644, 262, 1466, 1444, 340, 13, 314, 460, 3285, 9074, 13, 46606, 536]


In [15]:
enc_sample = enc_text[50:]

context_size = 4
x = enc_sample[ : context_size]
y = enc_sample[1 : context_size + 1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [16]:
for i in range(1, context_size + 1):
    context= enc_sample[:i]
    target = enc_sample[i]
    print(f"context: {context} -> target: {target}")

context: [290] -> target: 4920
context: [290, 4920] -> target: 2241
context: [290, 4920, 2241] -> target: 287
context: [290, 4920, 2241, 287] -> target: 257


In [17]:
for i in range(1, context_size + 1):
    context= enc_sample[:i]
    target = enc_sample[i]
    print(f"context: {tokenizer.decode(context)} -> target: {tokenizer.decode([target])}")


context:  and -> target:  established
context:  and established -> target:  himself
context:  and established himself -> target:  in
context:  and established himself in -> target:  a


In [18]:
import torch
from torch.utils.data import DataLoader, Dataset

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i : i + max_length]
            target_chunk = token_ids[i + 1 : i + max_length + 1]

            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [19]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128,
                        shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, 
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last, # 마지막 배치가 데이터 수와 일치하지 않으면 버림
        num_workers=num_workers # 전처리에 사용할 cpu 수
    )

    return dataloader

In [20]:
with open('The Verdict.txt', 'r') as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch[0]) # 입력 데이터
print(first_batch[1]) # 타겟 데이터

tensor([[  40,  367, 2885, 1464]])
tensor([[ 367, 2885, 1464, 1807]])


In [21]:
second_batch = next(data_iter)
print(second_batch[0])
print(second_batch[1])

tensor([[ 367, 2885, 1464, 1807]])
tensor([[2885, 1464, 1807, 3619]])


In [22]:
# 연습문제 2.2: 여러가지 스트라이드와 문맥 크기를 가진 데이터 로더 만들기
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=2, stride=2, shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print("max_length=2, stride=2")
print("first batch:")
print(first_batch[0])
print(first_batch[1])

second_batch = next(data_iter)
print("second batch:")
print(second_batch[0])
print(second_batch[1])




max_length=2, stride=2
first batch:
tensor([[ 40, 367]])
tensor([[ 367, 2885]])
second batch:
tensor([[2885, 1464]])
tensor([[1464, 1807]])


In [23]:
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=8, stride=2, shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print("max_length=8, stride=2")
print("first batch:")
print(first_batch[0])
print(first_batch[1])

second_batch = next(data_iter)
print("second batch:")
print(second_batch[0])
print(second_batch[1])

max_length=8, stride=2
first batch:
tensor([[  40,  367, 2885, 1464, 1807, 3619,  402,  271]])
tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899]])
second batch:
tensor([[ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138]])
tensor([[ 1464,  1807,  3619,   402,   271, 10899,  2138,   257]])


In [24]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, stride=4, shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(inputs)
print(targets)

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 토큰 임베딩 만들기

- 토큰 ID 를 임베딩 벡터로 변환 필요
- 역전파(backpropagation) 알고리즘으로 훈련하는 방법은 부록 A - A.4 절 참고
- `torch.nn.Embedding`: 원-핫 인코딩방식의 sparse 한 방식을 최적화해서 구현해놓은 것
  - 인덱스 룩업을 통해 진행

In [25]:
input_ids = torch.tensor([2, 3, 5, 1])
vocab_size = 6
output_dim = 3 # 임베딩 벡터 차원 (GPT-3 임베딩 크기는 12288)

torch.manual_seed(123) # 결과 재현을 위한 시드값 고정
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

print(embedding_layer.weight) # 6개의 토큰이 하나의 행에 할당, 열이 6개가 됨
print(embedding_layer(torch.tensor([3]))) # 토큰 3의 임베딩 벡터 출력
print(embedding_layer(input_ids))

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)
tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)
tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [26]:
from torchinfo import summary

summary(embedding_layer, input_data=input_ids)

Layer (type:depth-idx)                   Output Shape              Param #
Embedding                                [4, 3]                    18
Total params: 18
Trainable params: 18
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

## 2.8 단어 위치 인코딩 하기

- LLM 의 단점 중 하나는 시퀀스 안의 토큰 위치 또는 순서에 대한 개념이 셀프 어텐션 메커니즘에 없다는것?????? (3장 참조)
- deterministic(결정론적) 이라는 장점은 있지만,
  - LLM 의 셀프 어텐션 메커니즘 자체가 위치 구애받지 않기 때문에 LLM 에 추가적인 위치 정부를 주입하는 것이 도움됨... (뭔소리야)
- 어쨋든 위를 위해 두 종류의 위치를 고려한 임베딩을 사용할 수 있음
  - 상대 위치 임베딩
  - 절대 위치 임베딩: 시퀀스의 특정 위치에 직접 연관 됨
  

In [27]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [28]:
max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print(inputs)
print(inputs.shape)

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
torch.Size([8, 4])


In [29]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)
print(token_embeddings[0])

torch.Size([8, 4, 256])
tensor([[ 0.4913,  1.1239,  1.4588,  ..., -0.3995, -1.8735, -0.1445],
        [ 0.4481,  0.2536, -0.2655,  ...,  0.4997, -1.1991, -1.1844],
        [-0.2507, -0.0546,  0.6687,  ...,  0.9618,  2.3737, -0.0528],
        [ 0.9457,  0.8657,  1.6191,  ..., -0.4544, -0.7460,  0.3483]],
       grad_fn=<SelectBackward0>)


In [30]:
# 절대 위치 임베딩
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)
print(pos_embeddings)

torch.Size([4, 256])
tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       grad_fn=<EmbeddingBackward0>)


In [31]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)
print(input_embeddings[0])

torch.Size([8, 4, 256])
tensor([[ 2.2288,  0.5619,  0.8286,  ..., -0.6272, -0.2987,  0.8900],
        [ 2.0903, -0.4664, -0.0593,  ...,  0.9115, -1.0493, -1.6473],
        [-0.7158, -0.8304,  1.2494,  ...,  2.3952,  1.8773,  0.8051],
        [ 0.2703,  0.4029,  3.0514,  ...,  0.3595, -1.4548,  0.8310]],
       grad_fn=<SelectBackward0>)
